# OTC Treasury Reconciliation System
### Portfolio Project: Senior Treasury & Finance Operations
**Author:** Gilang Fajar Wijayanto  
**Date:** February 2026

---

## 1. Introduction
In a high-volume OTC (Over-The-Counter) trading environment, reconciling transactions across bank accounts, crypto wallets, and market maker statements is a critical control function. Errors in settlement matching lead to incorrect PnL recognition, tax leakage, and hidden liquidity risks.

This notebook demonstrates a **robust reconciliation engine** that:
1. Normalizes transaction data from disparate sources.
2. Pairs crypto and fiat settlement legs.
3. Recognizes PnL only upon dual-leg confirmation.
4. Flags discrepancies and settlement delays.

### System Architecture
![Architecture Flow](../diagrams/architecture_flow.png)


## 2. Setup and Data Loading
We use `pandas` for data manipulation and `matplotlib` for visualization. We'll load the synthetic dataset generated for FY 2024.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set aesthetic style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

# Define paths (local paths relative to repo)
TX_PATH = "../data/01_transactions.csv"
PNL_PATH = "../data/02_monthly_pnl.csv"
LEDGER_PATH = "../data/03_account_ledger.csv"

# Load datasets
df_tx = pd.read_csv(TX_PATH)
df_pnl = pd.read_csv(PNL_PATH)
df_ledger = pd.read_csv(LEDGER_PATH)

print(f"Loaded {len(df_tx)} transactions.")

## 3. Schema Walkthrough
Every row in the transaction dataset represents a single OTC trade. Key fields include:
- `idr_client_amount`: The total IDR the client pays or receives.
- `idr_mm_amount`: Our cost from the Market Maker.
- `tax_idr`: The 0.21% regulatory tax applied on the client amount.
- `net_pnl_idr`: Our final profit after tax (recognized only if settled).

In [ ]:
df_tx.info()
df_tx[['transaction_id', 'pair', 'direction', 'volume_crypto', 'client_price_idr', 'net_pnl_idr', 'status']].head()

## 4. Exploratory Analysis
Understanding the distribution of trading pairs and transaction statuses helps identify operational bottlenecks.

In [ ]:
# Overall transaction statistics
total_tx = len(df_tx)
settled_tx = len(df_tx[df_tx['status'] == 'SETTLED'])
pending_tx = len(df_tx[df_tx['status'] == 'PENDING'])
recon_tx = len(df_tx[df_tx['status'] == 'RECONCILING'])
failed_tx = len(df_tx[df_tx['status'] == 'FAILED'])
settlement_rate = (settled_tx / total_tx * 100)

print(f"=== Transaction Overview ===")
print(f"Total Transactions: {total_tx:,}")
print(f"Settled: {settled_tx:,} ({settlement_rate:.1f}%)")
print(f"Pending: {pending_tx:,}")
print(f"Reconciling: {recon_tx:,}")
print(f"Failed: {failed_tx:,}")
print()

# Status distribution visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart for pair distribution
pair_counts = df_tx['pair'].value_counts()
colors = ['#26a17b', '#2775ca', '#f7931a', '#d4a843']
ax1.pie(pair_counts, labels=pair_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)
ax1.set_title('Transaction Volume by Pair', fontsize=14, fontweight='bold')

# Bar chart for status distribution
status_counts = df_tx['status'].value_counts()
status_colors = {'SETTLED': '#26a17b', 'PENDING': '#fbbf24', 'RECONCILING': '#3b82f6', 'FAILED': '#ef4444'}
colors_list = [status_colors.get(s, '#6b7280') for s in status_counts.index]
ax2.bar(status_counts.index, status_counts.values, color=colors_list)
ax2.set_title('Transaction Status Breakdown', fontsize=14, fontweight='bold')
ax2.set_ylabel('Count')
ax2.set_xlabel('Status')
for i, v in enumerate(status_counts.values):
    ax2.text(i, v + 50, f'{v:,}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

### 4.1 Pair-Wise Performance Analysis
Different trading pairs contribute differently to overall profitability. Let's analyze PnL by pair.

In [ ]:
# Calculate PnL metrics by pair (settled transactions only)
settled_df = df_tx[df_tx['status'] == 'SETTLED'].copy()

pair_metrics = settled_df.groupby('pair').agg({
    'net_pnl_idr': 'sum',
    'gross_spread_idr': 'sum',
    'transaction_id': 'count',
    'spread_bps': 'mean'
}).round(2)

pair_metrics.columns = ['Net PnL (IDR)', 'Gross Spread (IDR)', 'Tx Count', 'Avg Spread (bps)']
pair_metrics = pair_metrics.sort_values('Net PnL (IDR)', ascending=False)

print("=== Pair-Wise Performance (Settled Only) ===")
print(pair_metrics)
print()
print(f"Total Net PnL: IDR {pair_metrics['Net PnL (IDR)'].sum():,.0f}")
print(f"Total Gross Spread: IDR {pair_metrics['Gross Spread (IDR)'].sum():,.0f}")
print()

# Visualize PnL contribution by pair
fig, ax = plt.subplots(figsize=(10, 6))
colors_map = {'USDT/IDR': '#26a17b', 'USDC/IDR': '#2775ca', 'BTC/IDR': '#f7931a', 'PAXG/IDR': '#d4a843'}
colors_list = [colors_map.get(p, '#6b7280') for p in pair_metrics.index]

bars = ax.barh(pair_metrics.index, pair_metrics['Net PnL (IDR)'] / 1e9, color=colors_list)
ax.set_xlabel('Net PnL (IDR Billions)', fontsize=12)
ax.set_title('Net PnL Contribution by Trading Pair', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(pair_metrics.iterrows()):
    ax.text(row['Net PnL (IDR)'] / 1e9 + 0.3, i, f"IDR {row['Net PnL (IDR)']/1e9:.1f}B", 
            va='center', fontweight='bold')

plt.tight_layout()
plt.show()

### 4.1.1 Understanding Spread Calculation

The **spread** is the difference between the price we quote to clients and the price we pay to market makers. It's measured in **basis points (bps)**, where 1 bps = 0.01%.

**Formula:**
```
Spread (bps) = ((Client Price - Market Maker Price) / Market Maker Price) × 10,000
```

**For BUY transactions (client buys crypto):**
- Client pays a higher IDR price than our MM cost
- Spread = (Client Price - MM Price) / MM Price × 10,000

**For SELL transactions (client sells crypto):**
- Client receives a lower IDR price than our MM sell price
- Spread = (MM Price - Client Price) / MM Price × 10,000

**Blended Spread:**
The overall average spread across all pairs, weighted by transaction volume. This is calculated as:
```
Blended Spread = Σ(Spread_i × Volume_i) / Σ(Volume_i)
```

Different pairs have different spread profiles based on liquidity, volatility, and market depth.

In [ ]:
# Calculate spread metrics by pair
print("=== Spread Analysis by Pair ===")
print()

for pair in settled_df['pair'].unique():
    pair_df = settled_df[settled_df['pair'] == pair]
    avg_spread = pair_df['spread_bps'].mean()
    min_spread = pair_df['spread_bps'].min()
    max_spread = pair_df['spread_bps'].max()
    std_spread = pair_df['spread_bps'].std()
    
    print(f"{pair}:")
    print(f"  Average Spread: {avg_spread:.1f} bps")
    print(f"  Range: {min_spread:.1f} - {max_spread:.1f} bps")
    print(f"  Std Dev: {std_spread:.1f} bps")
    print()

# Calculate volume-weighted blended spread
total_volume = settled_df['idr_client_amount'].sum()
weighted_spread = (settled_df['spread_bps'] * settled_df['idr_client_amount']).sum() / total_volume

print(f"Volume-Weighted Blended Spread: {weighted_spread:.1f} bps")
print(f"Simple Average Spread: {settled_df['spread_bps'].mean():.1f} bps")
print()

# Visualize spread distribution by pair
fig, ax = plt.subplots(figsize=(12, 6))
settled_df.boxplot(column='spread_bps', by='pair', ax=ax, patch_artist=True)
ax.set_title('Spread Distribution by Trading Pair', fontsize=14, fontweight='bold')
ax.set_xlabel('Trading Pair', fontsize=12)
ax.set_ylabel('Spread (bps)', fontsize=12)
plt.suptitle('')  # Remove the default title
ax.axhline(y=weighted_spread, color='red', linestyle='--', linewidth=2, label=f'Blended: {weighted_spread:.1f} bps')
ax.legend()
plt.tight_layout()
plt.show()

### 4.2 Monthly PnL Trend
Tracking monthly performance helps identify seasonality and operational consistency.

In [ ]:
# Monthly aggregation
settled_df['trade_date'] = pd.to_datetime(settled_df['trade_date'])
settled_df['month'] = settled_df['trade_date'].dt.strftime('%b')

monthly_pnl = settled_df.groupby('month').agg({
    'net_pnl_idr': 'sum',
    'gross_spread_idr': 'sum',
    'tax_idr': 'sum',
    'transaction_id': 'count'
})

# Reorder by calendar month
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_pnl = monthly_pnl.reindex(month_order)

print("=== Monthly Performance Summary ===")
print(monthly_pnl)
print()
print(f"Average Monthly Net PnL: IDR {monthly_pnl['net_pnl_idr'].mean():,.0f}")
print(f"Best Month: {monthly_pnl['net_pnl_idr'].idxmax()} (IDR {monthly_pnl['net_pnl_idr'].max():,.0f})")
print(f"Worst Month: {monthly_pnl['net_pnl_idr'].idxmin()} (IDR {monthly_pnl['net_pnl_idr'].min():,.0f})")
print()

# Visualize monthly trend
fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(monthly_pnl.index, monthly_pnl['net_pnl_idr'] / 1e9, color='#d4a843', edgecolor='black', linewidth=0.5)
ax.set_ylabel('Net PnL (IDR Billions)', fontsize=12)
ax.set_xlabel('Month', fontsize=12)
ax.set_title('Monthly Net PnL Trend (FY 2024)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
            f'{height:.1f}B', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

### 4.3 Top Clients by Net PnL
Understanding client concentration helps assess revenue diversification risk.

In [ ]:
# Top clients by Net PnL
client_pnl = settled_df.groupby('client_name')['net_pnl_idr'].sum().sort_values(ascending=False)
top_clients = client_pnl.head(8)

print("=== Top 8 Clients by Net PnL ===")
for i, (client, pnl) in enumerate(top_clients.items(), 1):
    pct = (pnl / client_pnl.sum() * 100)
    print(f"{i}. {client}: IDR {pnl:,.0f} ({pct:.1f}%)")

print()
top3_pct = (top_clients.head(3).sum() / client_pnl.sum() * 100)
print(f"Top 3 clients contribute: {top3_pct:.1f}% of total PnL")
print()

# Visualize top clients
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(range(len(top_clients)), top_clients.values / 1e9, color='#3b82f6')
ax.set_yticks(range(len(top_clients)))
ax.set_yticklabels(top_clients.index)
ax.set_xlabel('Net PnL (IDR Billions)', fontsize=12)
ax.set_title('Top 8 Clients by Net PnL Contribution', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, (client, pnl) in enumerate(top_clients.items()):
    ax.text(pnl / 1e9 + 0.1, i, f"IDR {pnl/1e9:.1f}B", va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Settlement Matching Logic
An OTC trade is only complete when both the **Crypto Leg** (wallet transfer) and the **Fiat Leg** (bank transfer) are confirmed. We track these via `crypto_settlement_timestamp` and `fiat_settlement_timestamp`.

In [ ]:
# Show dual-entry ledger structure for a single transaction
sample_tx_id = df_tx[df_tx['status'] == 'SETTLED']['transaction_id'].iloc[0]
df_ledger[df_ledger['transaction_id'] == sample_tx_id]

## 6. PnL Recognition Rule
**Rule:** PnL is recognized only when BOTH legs are settled. 
This prevents recognizing "paper profit" before the cash is actually in the bank or the crypto is in the wallet.

In [ ]:
# Convert to datetime
df_tx['trade_date'] = pd.to_datetime(df_tx['trade_date'])
df_tx['pnl_recognition_timestamp'] = pd.to_datetime(df_tx['pnl_recognition_timestamp'])

# Monthly Net PnL Waterfall
monthly_summary = df_tx[df_tx['status'] == 'SETTLED'].groupby('pnl_recognition_month')['net_pnl_idr'].sum()
monthly_summary.plot(kind='bar', title="Monthly Net PnL (IDR)", color='teal')
plt.show()

## 7. Discrepancy Detection
We flag transactions where the time gap between legs (e.g., bank transfer delay) exceeds the operational SLA (8 hours).

In [ ]:
df_tx['crypto_settlement_timestamp'] = pd.to_datetime(df_tx['crypto_settlement_timestamp'])
df_tx['fiat_settlement_timestamp'] = pd.to_datetime(df_tx['fiat_settlement_timestamp'])

df_tx['settlement_gap_hrs'] = (df_tx['fiat_settlement_timestamp'] - df_tx['crypto_settlement_timestamp']).dt.total_seconds() / 3600

# Flagging delays > 8 hours
delays = df_tx[df_tx['settlement_gap_hrs'] > 8][['transaction_id', 'pair', 'settlement_gap_hrs']]
print(f"Transactions with settlement delays > 8 hrs: {len(delays)}")
delays.head()

# Convert to datetime
df_tx['trade_date'] = pd.to_datetime(df_tx['trade_date'])
df_tx['pnl_recognition_timestamp'] = pd.to_datetime(df_tx['pnl_recognition_timestamp'])

# Calculate total volume and PnL metrics
total_volume = settled_df['idr_client_amount'].sum()
total_net_pnl = settled_df['net_pnl_idr'].sum()
total_gross_spread = settled_df['gross_spread_idr'].sum()
total_tax = settled_df['tax_idr'].sum()
avg_spread_bps = settled_df['spread_bps'].mean()

print("=== Overall Financial Metrics (Settled Transactions) ===")
print(f"Total Volume: IDR {total_volume:,.0f} ({total_volume/1e12:.1f}T)")
print(f"Gross Spread: IDR {total_gross_spread:,.0f} ({total_gross_spread/1e9:.1f}B)")
print(f"Tax Paid: IDR {total_tax:,.0f} ({total_tax/1e9:.1f}B)")
print(f"Net PnL: IDR {total_net_pnl:,.0f} ({total_net_pnl/1e9:.1f}B)")
print(f"Average Spread: {avg_spread_bps:.1f} bps")
print()
print(f"Effective PnL Margin: {(total_net_pnl / total_volume * 100):.3f}%")